# Dataset Preprocessing 

This notebook demonstrates the complete pipeline for:
1. Downloading the Edge-IIoT dataset from Kaggle
2. Loading and analyzing the datasets
3. Data cleaning and validation
4. Merging datasets and organizing by device
5. Converting data to streaming format
6. Exporting processed data for streaming

Dataset: [Edge-IIoT Set Dataset](https://www.kaggle.com/datasets/sibasispradhan/edge-iiotset-dataset)

The Edge-IIoT dataset contains sensor data from IoT edge devices for anomaly detection research.


**Preprocessing Kaggle Notebook Link:** [Preprocessing](https://www.kaggle.com/code/imedbenmadi/federated-learning-preprocessing)

## Install required packages

In [42]:
!pip install --quiet kaggle pandas numpy scipy scikit-learn kafka-python python-dotenv

In [43]:
import os
import json
import time
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np

os.environ['KAGGLE_CONFIG_DIR'] = os.path.expanduser('~/.kaggle')

print("Libraries imported successfully")


Libraries imported successfully


## 2. Download Dataset



In [44]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Use data folder for downloads
data_dir = Path('../data/raw')
data_dir.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {data_dir.absolute()}")

api = KaggleApi()
api.authenticate()
print("Kaggle API authenticated")


Data directory: c:\Users\imadb\OneDrive\Bureau\OST-2\notebooks\..\data\raw
Kaggle API authenticated


In [45]:
# Check if files already exist
csv_files = list(data_dir.glob('*.csv'))
if len(csv_files) >= 3:
    print("Dataset files already exist locally\n")
    print(f"Files in {data_dir}:")
    for file in sorted(csv_files):
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {file.name} ({size_mb:.2f} MB)")
else:
    print("Downloading Edge-IIoT dataset from Kaggle...")
    print("This may take several minutes...\n")
    
    api.dataset_download_files(
        'sibasispradhan/edge-iiotset-dataset',
        path=data_dir,
        unzip=True
    )
    
    print("Dataset downloaded successfully to data/raw/\n")
    print(f"Files in {data_dir}:")
    for file in sorted(data_dir.glob('*.csv')):
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  {file.name} ({size_mb:.2f} MB)")


Dataset files already exist locally

Files in ..\data\raw:
  DNN-EdgeIIoT-dataset.csv (1161.02 MB)
  live_data_training.csv (96.99 MB)
  ML-EdgeIIoT-dataset.csv (78.38 MB)


## 3. Load and Explore Datasets

Load all CSV files and analyze their structure and content.


In [46]:
csv_files = sorted(data_dir.glob('*.csv'))
print(f"Found {len(csv_files)} CSV files in {data_dir}\n")

datasets = {}
for csv_file in csv_files:
    print(f"Loading {csv_file.name}...")
    df = pd.read_csv(csv_file)
    datasets[csv_file.stem] = df
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {len(df.columns)}")
    print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")

print(f"Loaded {len(datasets)} datasets")


Found 3 CSV files in ..\data\raw

Loading DNN-EdgeIIoT-dataset.csv...Loading DNN-EdgeIIoT-dataset.csv...



C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\2785143168.py:7: DtypeWarning: Columns (2,3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


  Shape: (2219201, 63)
  Columns: 63
  Memory: 3330.01 MB

Loading live_data_training.csv...
  Memory: 3330.01 MB

Loading live_data_training.csv...


C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\2785143168.py:7: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


  Shape: (32481, 63)
  Columns: 63
  Memory: 120.53 MB

Loading ML-EdgeIIoT-dataset.csv...


C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\2785143168.py:7: DtypeWarning: Columns (3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


  Shape: (157800, 63)
  Columns: 63
  Memory: 235.52 MB

Loaded 3 datasets
  Memory: 235.52 MB

Loaded 3 datasets


In [47]:
for name, df in datasets.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {name}")
    print(f"{'='*60}")
    print(f"\nDimensions: {df.shape[0]} rows x {df.shape[1]} columns")
    print(f"\nData types:")
    print(df.dtypes)
    print(f"\nFirst few rows:")
    print(df.head())
    print(f"\nMissing values:")
    missing = df.isnull().sum()
    if missing.sum() == 0:
        print("None")
    else:
        print(missing[missing > 0])



Dataset: DNN-EdgeIIoT-dataset

Dimensions: 2219201 rows x 63 columns

Data types:
frame.time             object
ip.src_host            object
ip.dst_host            object
arp.dst.proto_ipv4     object
arp.opcode            float64
                       ...   
mbtcp.len             float64
mbtcp.trans_id        float64
mbtcp.unit_id         float64
Attack_label            int64
Attack_type            object
Length: 63, dtype: object

First few rows:
                  frame.time    ip.src_host    ip.dst_host arp.dst.proto_ipv4  \
0   2021 11:44:10.081753000   192.168.0.128  192.168.0.101                  0   
1   2021 11:44:10.162218000   192.168.0.101  192.168.0.128                  0   
2   2021 11:44:10.162271000   192.168.0.128  192.168.0.101                  0   
3   2021 11:44:10.162641000   192.168.0.128  192.168.0.101                  0   
4   2021 11:44:10.166132000   192.168.0.101  192.168.0.128                  0   

   arp.opcode  arp.hw.size arp.src.proto_ipv4  icmp.check

## 4. Data simple Cleaning

**Remove duplicates, handle missing values, and convert data types.**


In [56]:
def clean_dataset(df, name):
    """Clean and validate dataset"""
    print(f"Cleaning {name}...")
    
    initial_rows = len(df)
    
    # Remove duplicates
    df_clean = df.drop_duplicates().reset_index(drop=True)
    duplicates = initial_rows - len(df_clean)
    if duplicates > 0:
        print(f"  Removed {duplicates} duplicate rows")
    
    # Drop rows with any missing values
    rows_before = len(df_clean)
    df_clean = df_clean.dropna()
    missing_removed = rows_before - len(df_clean)
    if missing_removed > 0:
        print(f"  Removed {missing_removed} rows with missing values")
    
    # Convert object columns to numeric where possible
    converted_success = 0
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            try:
                converted = pd.to_numeric(df_clean[col], errors='coerce')
                if converted.notna().sum() > 0:
                    df_clean[col] = converted
                    converted_success += 1
            except:
                pass
    if converted_success > 0:
        print(f"  Converted {converted_success} object columns to numeric")
    
    # Add source dataset indicator
    df_clean['dataset_source'] = name
    
    print(f"  Final: {len(df_clean):,} rows ({100*len(df_clean)/initial_rows:.1f}% retained)\n")
    return df_clean

cleaned_datasets = {}
for name, df in datasets.items():
    cleaned_datasets[name] = clean_dataset(df, name)


Cleaning DNN-EdgeIIoT-dataset...
  Removed 815 duplicate rows
  Removed 815 duplicate rows
  Converted 19 object columns to numeric
  Final: 2,218,386 rows (100.0% retained)

Cleaning live_data_training...
  Converted 19 object columns to numeric
  Final: 2,218,386 rows (100.0% retained)

Cleaning live_data_training...
  Removed 77 duplicate rows
  Removed 77 duplicate rows
  Converted 6 object columns to numeric
  Final: 32,404 rows (99.8% retained)

Cleaning ML-EdgeIIoT-dataset...
  Converted 6 object columns to numeric
  Final: 32,404 rows (99.8% retained)

Cleaning ML-EdgeIIoT-dataset...
  Removed 814 duplicate rows
  Removed 814 duplicate rows
  Converted 19 object columns to numeric
  Final: 156,986 rows (99.5% retained)

  Converted 19 object columns to numeric
  Final: 156,986 rows (99.5% retained)



## 5. Select Dataset

Use the DNN-EdgeIIoT-dataset (largest dataset) for preprocessing.


In [57]:
print("Cleaned datasets:")
print(f"Total: {len(cleaned_datasets)}")
for key in cleaned_datasets.keys():
    print(f"  - {key}")


Cleaned datasets:
Total: 3
  - DNN-EdgeIIoT-dataset
  - live_data_training
  - ML-EdgeIIoT-dataset


In [58]:
# Select DNN-EdgeIIoT dataset
print("Selecting DNN-EdgeIIoT-dataset for preprocessing...\n")

df_merged = cleaned_datasets['DNN-EdgeIIoT-dataset'].copy()

print(f"Dataset selected: DNN-EdgeIIoT-dataset")
print(f"Final dataset shape: {df_merged.shape}")
print(f"Rows: {len(df_merged):,}")
print(f"Columns: {len(df_merged.columns)}")
print(f"\nDataset info:")
print(f"  Memory: {df_merged.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Selecting DNN-EdgeIIoT-dataset for preprocessing...

Dataset selected: DNN-EdgeIIoT-dataset
Final dataset shape: (2218386, 64)
Rows: 2,218,386
Columns: 64

Dataset info:
  Memory: 1347.80 MB
Dataset selected: DNN-EdgeIIoT-dataset
Final dataset shape: (2218386, 64)
Rows: 2,218,386
Columns: 64

Dataset info:
  Memory: 1347.80 MB


In [63]:
df_merged.head()

DNN-EdgeIIoT-dataset: Complete Structure Analysis

1. DATASET DIMENSIONS:
   Rows: 2,218,386
   Columns: 64
   Memory: 1347.80 MB

2. ALL COLUMNS (64 total):
    1. frame.time                | Type: float64    | Non-null: 400
    2. ip.src_host               | Type: float64    | Non-null: 285,125
    3. ip.dst_host               | Type: float64    | Non-null: 395,165
    4. arp.dst.proto_ipv4        | Type: float64    | Non-null: 2,200,999
    5. arp.opcode                | Type: float64    | Non-null: 2,218,386
    6. arp.hw.size               | Type: float64    | Non-null: 2,218,386
    7. arp.src.proto_ipv4        | Type: float64    | Non-null: 2,212,532
    8. icmp.checksum             | Type: float64    | Non-null: 2,218,386
    9. icmp.seq_le               | Type: float64    | Non-null: 2,218,386
   10. icmp.transmit_timestamp   | Type: float64    | Non-null: 2,218,386
   11. icmp.unused               | Type: float64    | Non-null: 2,218,386
   12. http.file_data            | Typ

In [60]:
df_merged

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type,dataset_source
0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Normal,DNN-EdgeIIoT-dataset
1,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,4.0,0.0,0.0,0.0,0,Normal,DNN-EdgeIIoT-dataset
2,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Normal,DNN-EdgeIIoT-dataset
3,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Normal,DNN-EdgeIIoT-dataset
4,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,24.0,0.0,0.0,0.0,0.0,0,Normal,DNN-EdgeIIoT-dataset
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2218381,NaN,NaN,NaN,0.0,0.0,0.0,0.0,31814.0,45620.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP,DNN-EdgeIIoT-dataset
2218382,NaN,NaN,NaN,0.0,0.0,0.0,0.0,27718.0,45636.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP,DNN-EdgeIIoT-dataset
2218383,NaN,NaN,NaN,0.0,0.0,0.0,0.0,18502.0,45672.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP,DNN-EdgeIIoT-dataset
2218384,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1862.0,45737.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DDoS_ICMP,DNN-EdgeIIoT-dataset


In [ ]:
# Remove columns that are entirely NaN (all missing values)
print("Removing columns with all missing values...\n")

initial_cols = len(df_merged.columns)

# Find columns that are completely NaN
all_nan_cols = df_merged.columns[df_merged.isnull().all()].tolist()

if len(all_nan_cols) > 0:
    print(f"Found {len(all_nan_cols)} columns with ALL missing values:")
    for col in all_nan_cols:
        print(f"  - {col}")
    
    # Drop them
    df_merged = df_merged.drop(columns=all_nan_cols)
    print(f"\n✓ Removed columns: {initial_cols} → {len(df_merged.columns)}")
else:
    print("✓ No columns with all missing values found")

print(f"\nFinal shape: {df_merged.shape}")

In [ ]:
# Check for existing device/identifier columns
print("Looking for device identifier columns...")
device_col_candidates = ['device_id', 'Device_ID', 'DeviceID', 'device', 'Device', 'id', 'ID', 'src_ip', 'dst_ip', 'ip.src', 'ip.dst']

device_col = None
for col in device_col_candidates:
    if col in df_merged.columns:
        # Check if column has meaningful values (not all NaN)
        if df_merged[col].notna().sum() > 0:
            device_col = col
            print(f"Found column: {col}")
            break

if device_col is None:
    print("No device identifier column found")
    print(f"Using dataset source + random grouping instead\n")
    
    # Group by dataset source and create device IDs within each
    num_devices = max(5, len(df_merged) // 1000)
    df_merged['device_id'] = df_merged.groupby('dataset_source').cumcount() % num_devices
    print(f"Created {num_devices} synthetic device IDs per dataset")
else:
    print(f"Using existing device column: {device_col}")
    df_merged.rename(columns={device_col: 'device_id'}, inplace=True)

print(f"\nDevices distribution:")
print(df_merged['device_id'].value_counts().sort_index())

print(f"\nDevice counts by dataset:")
print(df_merged.groupby(['dataset_source', 'device_id']).size().unstack(fill_value=0))


Looking for device identifier columns...
No device identifier column found
Using dataset source + random grouping instead

Created 2218 synthetic device IDs per dataset

Devices distribution:
device_id
0       1001
1       1001
2       1001
3       1001
4       1001
        ... 
2213    1000
2214    1000
2215    1000
2216    1000
2217    1000
Name: count, Length: 2218, dtype: int64

Device counts by dataset:
Created 2218 synthetic device IDs per dataset

Devices distribution:
device_id
0       1001
1       1001
2       1001
3       1001
4       1001
        ... 
2213    1000
2214    1000
2215    1000
2216    1000
2217    1000
Name: count, Length: 2218, dtype: int64

Device counts by dataset:
device_id             0     1     2     3     4     5     6     7     8     \
dataset_source                                                               
DNN-EdgeIIoT-dataset  1001  1001  1001  1001  1001  1001  1001  1001  1001   

device_id             9     ...  2208  2209  2210  2211  2212  2


## What needed to be done now 
- Categorical encoding that works: OneHotEncoder turns strings into numeric features and handles unseen categories; this prevents the “input must be uniformly strings or numbers” errors you saw after merges.
- NaN/type safety inside the model pipeline: Imputers and type casting run inside scikit-learn’s Pipeline/ColumnTransformer, so OneHotEncoder never receives mixed dtypes or NaNs.
- Scaling for numerics: Normalizes numeric ranges (without centering to save memory), which helps many models and stabilizes training.
- Consistent transforms for batches/streaming: The fitted pipeline object guarantees the same feature space across devices and future data (no column drift, no unknown-category crashes).
- Memory-safe feature space: Sparse one-hot, float32, and pruning of ultra high-cardinality columns avoid the 1.6 TiB dense blow-up that triggered your MemoryError.
- Reproducibility and portability: You save the fitted preprocessor (`preprocessor.pkl`) and sparse NPZ features so the server/clients can apply the exact same logic.


## 6. Feature Engineering & Preprocessing

Apply sklearn-based feature scaling and encoding using the preprocessing pipeline.
This ensures consistent preprocessing for model training and inference.


In [ ]:
print("Preprocessing modules ready")

Preprocessing modules ready


In [ ]:
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict, Any
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from scipy.sparse import issparse, save_npz

# Label column detection candidates
LABEL_GUESS_CANDIDATES = ["attack", "Attack", "Label", "label", "class", "Class", "Attack_type", "AttackType", "Category"]

def guess_label_column(df: pd.DataFrame, override: Optional[str] = None) -> str:
    """Auto-detect label column from common names"""
    if override and override in df.columns:
        return override
    for c in LABEL_GUESS_CANDIDATES:
        if c in df.columns:
            return c
    return df.columns[-1]

def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names: strip whitespace and replace spaces/dashes with underscores"""
    df = df.copy()
    df.columns = [str(c).strip().replace(" ", "_").replace("-", "_") for c in df.columns]
    return df

def to_binary_labels(y: pd.Series) -> np.ndarray:
    """Convert labels to binary: Benign/Normal/0 = 0, else = 1"""
    y_norm = y.astype(str).str.lower().str.strip()
    attack = ~(y_norm.isin(["benign", "normal", "0"]))
    return attack.astype(int).to_numpy()

def to_multiclass_labels(y: pd.Series) -> Tuple[np.ndarray, List[str]]:
    """Convert labels to multiclass with class names mapping""";
    classes, uniques = pd.factorize(y.astype(str).str.strip())
    return classes.astype(int), [str(u) for u in uniques]

@dataclass
class FittedPreprocessor:
    """Container for fitted preprocessing pipeline and metadata"""
    pipeline: Pipeline
    features: List[str]
    label_name: str
    classes: Optional[List[str]] = None

def _detect_feature_columns(X: pd.DataFrame) -> Tuple[List[str], List[str]]:
    """Detect numeric vs categorical columns based on dtype."""
    numeric_cols = X.select_dtypes(include=["number", "float", "int", "Int64", "Float64"]).columns.tolist()
    cat_cols = [c for c in X.columns if c not in numeric_cols]
    return numeric_cols, cat_cols

def _cast_to_str(X):
    """Cast all columns to string for categorical pipeline."""
    if isinstance(X, pd.DataFrame):
        return X.astype(str)
    else:
        return X.astype(str)

def build_pipeline(df: pd.DataFrame, label_col: str) -> Tuple[Pipeline, List[str], List[str], List[str]]:
    """Simple, memory-friendly pipeline:
    - Exclude reserved identifiers from features
    - Numeric: SimpleImputer(mean)
    - Categorical: Fill NaN with 'missing' -> Cast to string -> OneHotEncoder(sparse, float32)
    """
    X = df.drop(columns=[label_col])

    # Exclude identifier-like columns from features (used for grouping/metadata)
    reserved = {"device_id", "dataset_source"}
    X = X.drop(columns=[c for c in reserved if c in X.columns], errors='ignore')

    # Drop columns that are completely empty (all NaN)
    X = X.dropna(axis=1, how='all')

    numeric_cols, cat_cols = _detect_feature_columns(X)

    numeric_pipeline = Pipeline(steps=[
        ("impute", SimpleImputer(strategy="mean")),
    ])

    # OneHotEncoder with safe fallbacks for different sklearn versions
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=np.float32, min_frequency=0.01)
    except TypeError:
        try:
            ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True, dtype=np.float32, max_categories=50)
        except TypeError:
            try:
                ohe = OneHotEncoder(handle_unknown="ignore", sparse=True, dtype=np.float32)
            except TypeError:
                ohe = OneHotEncoder(handle_unknown="ignore")

    categorical_pipeline = Pipeline(steps=[
        ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
        ("cast_str", FunctionTransformer(_cast_to_str, validate=False)),
        ("ohe", ohe),
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric_pipeline, numeric_cols),
            ("cat", categorical_pipeline, cat_cols),
        ],
        remainder="drop",
    )

    pipe = Pipeline([("pre", pre)])
    return pipe, X.columns.tolist(), numeric_cols, cat_cols

# Legacy single-shot fit/transform (kept for compatibility)
def fit_transform_first_batch(df: pd.DataFrame, task_mode: str = "binary", label_override: Optional[str] = None
                             ) -> Tuple[Any, np.ndarray, FittedPreprocessor]:
    df = clean_column_names(df)
    label_col = guess_label_column(df, label_override)
    y_raw = df[label_col]

    if task_mode.lower().startswith("multi"):
        y, classes = to_multiclass_labels(y_raw)
    else:
        y = to_binary_labels(y_raw)
        classes = None

    pipe, features, *_ = build_pipeline(df, label_col)
    X = pipe.fit_transform(df.drop(columns=[label_col]))
    # Cast to float32 if dense
    if hasattr(X, 'astype') and not issparse(X):
        X = X.astype("float32")
    return X, y.astype("int64"), FittedPreprocessor(pipeline=pipe, features=features,
                                                     label_name=label_col, classes=classes)

# New: memory-safe chunked pipeline

def fit_on_sample_and_transform_in_chunks(
    df: pd.DataFrame,
    task_mode: str = "binary",
    label_override: Optional[str] = None,
    sample_n: int = 200_000,
    chunk_size: int = 100_000,
    out_dir: Optional[Path] = None,
) -> Tuple[FittedPreprocessor, Dict[str, Any]]:
    """Fit on a sample, then transform the full dataset in chunks and write to disk.
    Returns (preprocessor, metadata) where metadata has total_samples, n_features, chunks info.
    """
    df = clean_column_names(df)
    label_col = guess_label_column(df, label_override)

    # Fit on a sample (features only)
    n_rows = len(df)
    fit_idx = np.random.RandomState(42).choice(n_rows, size=min(sample_n, n_rows), replace=False)
    df_fit = df.iloc[fit_idx]

    pipe, features, *_ = build_pipeline(df, label_col)
    pipe.fit(df_fit.drop(columns=[label_col]))

    fitted = FittedPreprocessor(pipeline=pipe, features=features, label_name=label_col, classes=None)

    # Prepare output
    out_dir = Path('../data/processed/chunks') if out_dir is None else out_dir
    out_dir.mkdir(parents=True, exist_ok=True)

    total = n_rows
    n_features = None
    chunk_files = []
    total_y = 0

    for start in range(0, total, chunk_size):
        end = min(start + chunk_size, total)
        df_chunk = df.iloc[start:end]
        X_chunk = pipe.transform(df_chunk.drop(columns=[label_col]))
        # Determine n_features from first chunk
        if n_features is None:
            n_features = X_chunk.shape[1]
        # Labels (binary by default)
        if task_mode.lower().startswith("multi"):
            y_chunk, classes = to_multiclass_labels(df_chunk[label_col])
            if fitted.classes is None:
                fitted.classes = classes
        else:
            y_chunk = to_binary_labels(df_chunk[label_col])

        # Save chunk
        xi = start // chunk_size
        X_path = out_dir / f"X_chunk_{xi}.npz"
        y_path = out_dir / f"y_chunk_{xi}.npy"
        if issparse(X_chunk):
            save_npz(X_path, X_chunk)
        else:
            np.savez(X_path, X=X_chunk.astype(np.float32))
        np.save(y_path, y_chunk.astype(np.int64))
        chunk_files.append({"X": str(X_path), "y": str(y_path), "rows": int(len(y_chunk))})
        total_y += len(y_chunk)
        if xi % 5 == 0:
            print(f"  Saved chunks up to index {xi} ({end}/{total} rows)")

    meta = {
        "total_samples": int(total_y),
        "n_features": int(n_features) if n_features is not None else None,
        "label_name": label_col,
        "out_dir": str(out_dir),
        "chunks": chunk_files,
    }
    return fitted, meta

print("Preprocessing utilities loaded successfully (chunked mode available)")


Preprocessing utilities loaded successfully (chunked mode available)


In [ ]:
print("Applying feature preprocessing to merged dataset (chunked, memory-safe)\n")

# Use chunked pipeline to avoid large in-memory matrices
# - Fits on a sample of the merged data
# - Transforms the entire dataset in chunks and writes NPZ/NPY files

preprocessor, preprocess_meta = fit_on_sample_and_transform_in_chunks(
    df=df_merged,              # no .copy() to avoid extra memory
    task_mode="binary",       # change to 'multiclass' if needed
    label_override=None,
    sample_n=200_000,
    chunk_size=100_000,
)

print("\nPreprocessing Results (chunked):")
print(f"  Total rows processed: {preprocess_meta['total_samples']:,}")
print(f"  Output feature dimension: {preprocess_meta['n_features']}")
print(f"  Label column detected: '{preprocessor.label_name}'")
print(f"  Chunks written to: {preprocess_meta['out_dir']}")
print(f"  Number of chunks: {len(preprocess_meta['chunks'])}")
if len(preprocess_meta['chunks']):
    print(f"  First chunk example: {preprocess_meta['chunks'][0]}")

# For compatibility with later cells (avoid using giant arrays in-memory)
X_processed = None
y_processed = None
print("\nPreprocessed dataset saved to disk as chunks; continue with device preprocessing and export.")


Applying feature preprocessing to merged dataset (chunked, memory-safe)



C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))


  Saved chunks up to index 0 (100000/2407776 rows)


C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local

  Saved chunks up to index 5 (600000/2407776 rows)


...

  Saved chunks up to index 15 (1600000/2407776 rows)


C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local

  Saved chunks up to index 20 (2100000/2407776 rows)


C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))
C:\Users\imadb\AppData\Local\Temp\ipykernel_24536\678152994.py:188: RuntimeWarning: overflow encountered in cast
  np.savez(X_path, X=X_chunk.astype(np.float32))



Preprocessing Results (chunked):
  Total rows processed: 2,407,776
  Output feature dimension: 72
  Label column detected: 'Attack_type'
  Chunks written to: ..\data\processed\chunks
  Number of chunks: 25
  First chunk example: {'X': '..\\data\\processed\\chunks\\X_chunk_0.npz', 'y': '..\\data\\processed\\chunks\\y_chunk_0.npy', 'rows': 100000}

Preprocessed dataset saved to disk as chunks; continue with device preprocessing and export.


## 7. Group by Device

Organize the dataset by device for streaming preparation.


In [ ]:
print("Grouping data by device...")

device_groups = {}
for device_id, group in df_merged.groupby('device_id'):
    device_groups[str(device_id)] = group.reset_index(drop=True)

print(f"Created {len(device_groups)} device groups\n")

print("Device group statistics:")
print("-" * 60)
for device_id, group in sorted(device_groups.items()):
    print(f"Device {device_id}: {len(group)} rows")

print(f"\nData grouped by device")


Grouping data by device...
Created 2407 device groups

Device group statistics:
------------------------------------------------------------
Device 0: 1002 rows
Device 1: 1002 rows
Device 10: 1002 rows
Device 100: 1002 rows
Device 1000: 1001 rows
Device 1001: 1001 rows
Device 1002: 1001 rows
Device 1003: 1001 rows
Device 1004: 1001 rows
Device 1005: 1001 rows
Device 1006: 1001 rows
Device 1007: 1001 rows
Device 1008: 1001 rows
Device 1009: 1001 rows
Device 101: 1002 rows
Device 1010: 1001 rows
Device 1011: 1001 rows
Device 1012: 1001 rows
Device 1013: 1001 rows
Device 1014: 1001 rows
Device 1015: 1001 rows
Device 1016: 1001 rows
Device 1017: 1001 rows
Device 1018: 1001 rows
Device 1019: 1001 rows
Device 102: 1002 rows
Device 1020: 1001 rows
Device 1021: 1001 rows
Device 1022: 1001 rows
Device 1023: 1001 rows
Device 1024: 1001 rows
Device 1025: 1001 rows
Device 1026: 1001 rows
Device 1027: 1001 rows
Device 1028: 1001 rows
Device 1029: 1001 rows
Device 103: 1002 rows
Device 1030: 1001 ro

## 8. Export Results

Save preprocessed data and statistics.


In [ ]:
from pathlib import Path
import json

out_dir = Path('../data/processed')
chunks_dir = out_dir / 'chunks'

print("PREPROCESSING EXPORT SUMMARY\n")
print("=" * 60)

# Check what files exist
if out_dir.exists():
    merged_csv = out_dir / 'merged_data.csv'
    preprocessor_pkl = out_dir / 'preprocessor.pkl'
    summary_json = out_dir / 'processing_summary.json'
    
    files = []
    if merged_csv.exists():
        files.append(f"✓ merged_data.csv ({merged_csv.stat().st_size / 1024**2:.1f} MB)")
    if preprocessor_pkl.exists():
        files.append(f"✓ preprocessor.pkl")
    if chunks_dir.exists():
        chunks = list(chunks_dir.glob('X_chunk_*.npz'))
        if chunks:
            files.append(f"✓ Preprocessed chunks: {len(chunks)} files")
    if summary_json.exists():
        files.append(f"✓ processing_summary.json")
    
    for f in files:
        print(f)

print("\n" + "=" * 60)
print(f"Output directory: {out_dir.absolute()}")
print("\n✓ Preprocessing complete!")


PREPROCESSING EXPORT SUMMARY

✓ merged_data.csv (671.0 MB)
✓ Preprocessed chunks: 25 files
✓ processing_summary.json

Output directory: c:\Users\imadb\OneDrive\Bureau\OST-2\notebooks\..\data\processed

✓ Preprocessing complete!
